In [ ]:
# Index TMDB Corpus
from time import perf_counter 
from ltr.client.solr_client import SolrClient
import json
client = SolrClient(host='http://localhost:8983/solr')

from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies
movies=indexable_movies(movies='data/tmdb_ai_pow_search.json')
rebuild(client, index='tmdb', doc_src=movies)

In [ ]:
def judg_csv(judgment):
    from ltr.helpers.movies import get_movie
    judgedMovie = get_movie(movies='data/tmdb_ai_pow_search.json', tmdb_id=judgment.doc_id)
    return "{grade},'{title}',{keywords}".format(grade=judgment.grade,
                                                 title=judgedMovie['title'],
                                                 keywords=judgment.keywords)



In [ ]:
from ltr.client.solr_client import SolrClient

client = SolrClient(host='http://localhost:8983/solr')

In [ ]:
from ltr.judgments import Judgment

Judgment(grade=1, keywords='social network', doc_id=37799)

In [ ]:

mini_judg_list=[
    # for 'social network' query
    Judgment(grade=1, keywords='social network', doc_id='37799'), #The Social Network
    Judgment(grade=0, keywords='social network', doc_id='267752'), # #chicagoGirl
    Judgment(grade=0, keywords='social network', doc_id='38408'), # Life As We Know It
    Judgment(grade=0, keywords='social network', doc_id='28303'), # The Cheyenne Social Club
    
    # for 'star wars' query
    Judgment(grade=1, keywords='star wars', doc_id='11'), # star wars
    Judgment(grade=1, keywords='star wars', doc_id='1892'), # return of jedi
    Judgment(grade=0, keywords='star wars', doc_id='54138'),# Star Trek Into Darkness
    Judgment(grade=0, keywords='star wars', doc_id='85783'), # The Star
    Judgment(grade=0, keywords='star wars', doc_id='325553'), # Battlestar Galactica
]

mini_judg_list

In [ ]:
from ltr.judgments import judgments_open

with judgments_open('data/dummy_judgments.txt', 'w') as judgment_writer:
    for j in mini_judg_list:
        judgment_writer.write(j)

print(open('data/dummy_judgments.txt').read())

In [ ]:
mini_judg_list[0].features


In [ ]:
import requests

feature_set = [
    {
      "name" : "title_phrase_bm25",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { #q=title:({$keywords})
        "q" : "title:(${keywords})"
      }
    },
    {
      "name" : "overview_bm25",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "overview:(${keywords})"
      }
    },
    {
      "name" : "release_year",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "{!func}release_year"
}}]

resp = requests.put('http://localhost:8983/solr/tmdb/schema/feature-store',
                    json=feature_set)
resp.text

In [ ]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='movies')
for qid, query_judgments in groupby(mini_judg_list, key=lambda j: j.qid):
    ftr_logger.log_for_qid(judgments=query_judgments, 
                           qid=qid)
        
ftr_logger.logged

In [ ]:
import requests

logging_solr_query = {
    "fl": "id,title,[features store=movies efi.keywords=\"social network\"]",
    'q': "id:37799 OR id:267752 id:38408 OR id:28303", #social network graded documents
    'rows': 10,
    'wt': 'json'  
}

resp = requests.post('http://localhost:8983/solr/tmdb/select',
                     data=logging_solr_query)

resp.json()

# The following code is omitted from the book

In [ ]:

mini_judg_list=[
    # for 'social network' query
    Judgment(grade=1, keywords='social network', doc_id='37799'), #The Social Network
    Judgment(grade=0, keywords='social network', doc_id='267752'), # #chicagoGirl
    Judgment(grade=0, keywords='social network', doc_id='38408'), # Life As We Know It
    Judgment(grade=0, keywords='social network', doc_id='28303'), # The Cheyenne Social Club
    
    # for 'star wars' query
    Judgment(grade=1, keywords='star wars', doc_id='11'), # star wars
    Judgment(grade=1, keywords='star wars', doc_id='1892'), # return of jedi
    Judgment(grade=0, keywords='star wars', doc_id='54138'),# Star Trek Into Darkness
    Judgment(grade=0, keywords='star wars', doc_id='85783'), # The Star
    Judgment(grade=0, keywords='star wars', doc_id='325553'), # Battlestar Galactica
]

mini_judg_list

# Save off features for qid=1
CURR_QID=1

solr_json = resp.json()
doc_id_to_features = {}

# Map Doc Id => Features
for doc in solr_json['response']['docs']:
    # Parse '[features] array', ie
    # title_bm25=0.0,overview_bm25=13.237938,vote_average=7.0'
    features = doc['[features]']
    features = features.split(',')
    features = [float(ftr.split('=')[1]) for ftr in features]
    
    doc_id_to_features[doc['id']] = features

# Save in correct judgment
for judgment in mini_judg_list:
    if judgment.qid == CURR_QID:
        try:
            judgment.features = doc_id_to_features[judgment.doc_id]
        except KeyError:
            pass

mini_judg_list

In [ ]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='movies')
for qid, query_judgments in groupby(mini_judg_list, key=lambda j: j.qid):
    ftr_logger.log_for_qid(judgments=query_judgments, 
                           qid=qid)
        
ftr_logger.logged

In [ ]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby
from ltr import download

#judgments='http://es-learn-to-rank.labs.o19s.com/title_judgments_binary.txt'
#download([judgments], dest='data/')

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='movies')

with judgments_open('data/ai_pow_search_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(judgments=query_judgments, 
                               qid=qid,
                               keywords=judgment_list.keywords(qid))
    
ftr_logger.logged

In [ ]:
from ltr.judgments import judgments_to_nparray
import numpy as np 
import pylab as pl
import matplotlib as mpl


features, predictors=judgments_to_nparray(ftr_logger.logged)
features.shape

In [ ]:
# Crocodile Dundee and Rocky have nice Linear Shapes
# TODO -> pull in larger judgment list (that has social network, etc...)

from sklearn.linear_model import LinearRegression

norm = mpl.colors.Normalize(0,1.0)

def draw_linear_fit(x, y):
    linear_regression = LinearRegression()
    linear_regression.fit(x, y)
    title_coef = np.asscalar(linear_regression.coef_[0])
    over_coef = np.asscalar(linear_regression.coef_[1])
    intercept = linear_regression.intercept_
    print(title_coef,over_coef,intercept)
    ax = pl.axes()
    ax.arrow(0, intercept, 150*title_coef, 150*over_coef, head_width=0.45, 
             head_length=0.7, fc='black', ec='black')
    return title_coef, over_coef


def plot_all(predictors):
    qids=predictors[:,1]
    qidA=np.argwhere(qids==qid).ravel()

    x_qidA = features
    x_qidA
    y_qidA = predictors[:, 0]
    pl.scatter(x_qidA[:,0], x_qidA[:,1], c=y_qidA, marker='*')
    
    draw_linear_fit(x=x_qidA[:,:2], y=y_qidA)
    
    pl.title("Relevances for all".format())
    pl.xlabel(xlabel="Title Phrase BM25")
    pl.ylabel(ylabel="Overview BM25")


def plot_qids(qids, predictors):
    qid_col=predictors[:,1]
    qid_idxs=np.array([])
    kws = []
    for qid in qids:
        qid_idxs=np.append(qid_idxs, np.argwhere(qid_col==qid).ravel()).astype(int)
        judgment=ftr_logger.logged[np.asscalar(qid_idxs[-1])]
        kws.append(judgment.keywords)
    x_qidA = features[qid_idxs]
    x_qidA
    y_qidA = predictors[qid_idxs, 0]
    pl.scatter(x_qidA[:,0], x_qidA[:,1], c=y_qidA, marker='*')
    
    title_coef, overview_coef = draw_linear_fit(x=x_qidA[:,:2], y=y_qidA)
    
    pl.title("Relevances for q={:.20}\n coef_ti={:1.3f}, coef_ov={:1.3f}"
             .format(",".join(kws), title_coef, overview_coef))
    pl.xlabel(xlabel="Title Phrase BM25")
    pl.ylabel(ylabel="Overview BM25")
    
def plot_qids2(qids, predictors, focus=None):
    from ltr.helpers.movies import get_movie
    
    if focus is None:
        focus=qids
    
    from random import shuffle
    from itertools import product
    r = list(range(0,5,1)); shuffle(r)
    g = list(range(0,5,1)); shuffle(g)
    b = list(range(0,5,1)); shuffle(b)
    
    out_of_focus_alpha=0.1
    in_focus_alpha=0.9
    
    if len(qids) > 3:
        # Make a random set of colors per query
        colors = [[r*0.1,g*0.1,b*0.1,out_of_focus_alpha] for r,g,b in product(r,g,b)]
        shuffle(colors)
    else: # These are intentionally looking different
        max_c = 0.4
        colors = [[0,max_c,0,out_of_focus_alpha],
                  [max_c,0,0,out_of_focus_alpha],
                  [0,0,max_c,out_of_focus_alpha]]

    qid_col=predictors[:,1]
    qid_idxs=np.array([])
    kws = []
    markers=('.', 'P') # Negative / Positive relevance markers...
    legend_paths=[]
    legend_labels=[]
    for idx, qid in enumerate(qids):
        qid_idxs=np.argwhere(qid_col==qid).ravel().astype(int)
        judgment=ftr_logger.logged[np.asscalar(qid_idxs[-1])]
        kws.append(judgment.keywords)
        x_qidA = features[qid_idxs]
        x_qidA
        y_qidA = predictors[qid_idxs, 0] 
        color = colors[idx]
        if qid in focus:
            #title_coef, overview_coef = draw_linear_fit(x=x_qidA[:,:2], y=y_qidA)
            color[3] = in_focus_alpha
        for grade in [1,0]:
            this_grade=np.argwhere(y_qidA==grade)
            path = pl.scatter(x_qidA[this_grade,0], 
                              x_qidA[this_grade,1], 
                               marker=markers[grade], 
                               facecolors=color,
                               edgecolors=color,
                               norm=norm)
            legend_paths.append(path)
            if grade == 0:
                legend_labels.append(judgment.keywords + " irrelevant")
            else:
                legend_labels.append(judgment.keywords + " relevant")

        
    
    pl.title("Relevances for keywords: {:.25}".format(",".join(kws)))
    pl.xlabel(xlabel="Title Phrase BM25")
    pl.ylabel(ylabel="Overview BM25")
    pl.legend(legend_paths, legend_labels, loc='lower center',
              bbox_to_anchor=[0.5,-0.5])
    pl.savefig('fig.png', dpi=300)

#plot_all(predictors)

In [ ]:
plot_qids2(qids=[11,40], focus=[11,40], predictors=predictors)

In [ ]:
plot_all(predictors=predictors)